In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys
from pathlib import Path
import re 

import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid

root_path = Path('/dev/projects/cds/')

if (path := str(root_path.resolve())) not in sys.path:
    sys.path.append(path)

from api.blp import BlpQuery
from api.data.base import data_path
from api.data.curves import curves

In [3]:
# Technically, the model requires the discount factor to be 1 at spot;
# but we can't do that and also have the discount curve extend back to
# the trade date.  For the time being, we'll keep discount = 1 at trade.
# The results match anyway.

#### Settings

In [100]:
cds_name = "ITALY CDS USD SR 5Y D14 Curncy"
trade_date = pd.Timestamp('2022-06-24') # pd.Timestamp.today().floor('d')
curve_name = 'USD ISDA CDS SOFR Rates Swap Curve'

bdh_options = {
    'calendarCodeOverride': '5D',
    "nonTradingDayFillOption": "ALL_CALENDAR_DAYS",
    "nonTradingDayFillMethod": "PREVIOUS_VALUE"
}

dt_fmt = '%Y-%m-%d'
bbg_dt_fmt = '%Y%m%d'

In [101]:
### Bloomberg Connection
bq = BlpQuery(timeout=50000).start()

#### Local data

In [102]:
curve_hist = pd.read_csv(data_path / 'curve_history.csv', parse_dates=['date'], dayfirst=True)
cds_refs = pd.read_csv(data_path / 'cds_reference.csv', parse_dates=['maturity'], dayfirst=True)

#### Bloomberg data

In [103]:
cds_price = bq.bdh(
    securities=[cds_name],
    fields=["px_last"],
    start_date=trade_date.strftime(bbg_dt_fmt),
    end_date=trade_date.strftime(bbg_dt_fmt),
    options=bdh_options
).rename({'px_last': 'last_price'}, axis=1).squeeze()

In [104]:
cds_ref = bq.bdp(
    securities=[cds_name],
    fields=[
        "maturity",
        "cds_recovery_rt"
    ],
).rename({'cds_recovery_rt': 'recovery'}, axis=1).squeeze()

#### Data parsing

In [105]:
curve = [curve for curve in curves if curve['name'] == curve_name][0]
curve_hist_ = curve_hist.query(f'curve_name == "{curve["name"]}"')
curve_hist_ = curve_hist_.assign(
    tenor=pd.Categorical(
        curve_hist_["tenor"],
        categories=[memb['tenor'] for memb in curve['members']],
        ordered=True
    )
)
curve_hist_date_ = curve_hist_.query(f'date == "{trade_date:{dt_fmt}}"').sort_values('tenor')

In [106]:
curve_hist_date_

date         security  last_price tenor  \
3976 2022-06-24   USOSFRA Curncy      1.4909    1M   
3364 2022-06-24   USOSFRB Curncy      1.8589    2M   
1528 2022-06-24   USOSFRC Curncy      2.0050    3M   
3058 2022-06-24   USOSFRF Curncy      2.5480    6M   
5200 2022-06-24   USOSFR1 Curncy      3.0250    1Y   
4588 2022-06-24   USOSFR2 Curncy      3.0878    2Y   
5506 2022-06-24   USOSFR3 Curncy      2.9861    3Y   
1834 2022-06-24   USOSFR4 Curncy      2.9210    4Y   
1222 2022-06-24   USOSFR5 Curncy      2.8891    5Y   
5812 2022-06-24   USOSFR6 Curncy      2.8701    6Y   
2752 2022-06-24   USOSFR7 Curncy      2.8592    7Y   
2140 2022-06-24   USOSFR8 Curncy      2.8552    8Y   
4894 2022-06-24   USOSFR9 Curncy      2.8592    9Y   
2446 2022-06-24  USOSFR10 Curncy      2.8712   10Y   
3670 2022-06-24  USOSFR12 Curncy      2.9012   12Y   
4282 2022-06-24  USOSFR15 Curncy      2.9273   15Y   
304  2022-06-24  USOSFR20 Curncy      2.8794   20Y   
916  2022-06-24  USOSFR25 Curncy      2.7655   25Y   
610  2022-06-24  USOSFR30 Curncy      2.6565   30Y   

                              curve_name  
3976  USD ISDA CDS SOFR Rates Swap Curve  
3364  USD ISDA CDS SOFR Rates Swap Curve  
1528  USD ISDA CDS SOFR Rates Swap Curve  
3058  USD ISDA CDS SOFR Rates Swap Curve  
5200  USD ISDA CDS SOFR Rates Swap Curve  
4588  USD ISDA CDS SOFR Rates Swap Curve  
5506  USD ISDA CDS SOFR Rates Swap Curve  
1834  USD ISDA CDS SOFR Rates Swap Curve  
1222  USD ISDA CDS SOFR Rates Swap Curve  
5812  USD ISDA CDS SOFR Rates Swap Curve  
2752  USD ISDA CDS SOFR Rates Swap Curve  
2140  USD ISDA CDS SOFR Rates Swap Curve  
4894  USD ISDA CDS SOFR Rates Swap Curve  
2446  USD ISDA CDS SOFR Rates Swap Curve  
3670  USD ISDA CDS SOFR Rates Swap Curve  
4282  USD ISDA CDS SOFR Rates Swap Curve  
304   USD ISDA CDS SOFR Rates Swap Curve  
916   USD ISDA CDS SOFR Rates Swap Curve  
610   USD ISDA CDS SOFR Rates Swap Curve

In [107]:
deposit_tenors = [
    '1M', 
    '2M', 
    '3M', 
    '6M', 
    '1Y'
]
swap_tenors = [
    '2Y', 
    '3Y', 
    '4Y', 
    '5Y', 
    '6Y',
    '7Y',
    '8Y',
    '9Y',
    '10Y',
    '12Y',
    '15Y',
    '20Y',
    '25Y',
    '30Y'
]
deposit_data = curve_hist_date_.query(f"tenor in {deposit_tenors}").set_index('tenor')['last_price']
swap_data = curve_hist_date_.query(f"tenor in {swap_tenors}").set_index('tenor')['last_price']

#### QuantLib

In [108]:
def get_ql_date(date):
    return ql.Date(date.day, date.month, date.year)

In [116]:
def get_term_date(date):
    y, m, d = date.year, date.month, date.day
    day = 20
    if date < pd.Timestamp(year=y, month=3, day=20):
        year = y - 1
        month = 12
    elif date < pd.Timestamp(year=y, month=9, day=20):
        year = y
        month = 6
    else:
        year = y
        month = 12
    start = pd.Timestamp(year=year, month=month, day=day)
    return start + pd.tseries.offsets.DateOffset(years=5)

In [117]:
# date = ql.Date(20, 6, 2022)
ql_trade_date = get_ql_date(trade_date)
ql_term_date = get_ql_date(get_term_date(trade_date))
ql_upfront_date = ql.WeekendsOnly().advance(ql_trade_date, 3*ql.Period(ql.Daily))
spot_date = ql.WeekendsOnly().advance(ql_trade_date, 2*ql.Period(ql.Daily))
period_parser = ql.PeriodParser()
ql.Settings.instance().setEvaluationDate(ql_trade_date)

In [119]:
deposit_rate_helpers = [
    ql.DepositRateHelper(
        rate / 100,  # rate
        period_parser.parse(tenor),  # tenor                                      
        2,  # fixingDays 
        ql.WeekendsOnly(),
        ql.ModifiedFollowing,
        False, 
        ql.Actual360()
    ) for tenor, rate in deposit_data.items()
]

In [120]:
isda_ibor = ql.IborIndex(
    'IsdaIbor',  # familyName
    3*ql.Period(ql.Monthly),  # tenor 
    2,  # settlementDays
    ql.USDCurrency(),  # currency
    ql.WeekendsOnly(),  # fixingCalendar
    ql.ModifiedFollowing, # convention
    False, # endOfMonth
    ql.Actual360(),  # dayCounter
)

In [121]:
swap_rate_helpers = [
    ql.SwapRateHelper(
        rate / 100,  # rate
        period_parser.parse(tenor),  # tenor
        ql.WeekendsOnly(),  # calendar
        ql.Semiannual,  # fixedFrequency
        ql.ModifiedFollowing,  # fixedConvention
        ql.Thirty360(),  # fixedDayCount
        isda_ibor,  # iborIndex
    ) for tenor, rate in swap_data.items()
]

In [122]:
isda_rate_helpers = [*deposit_rate_helpers, *swap_rate_helpers]
rate_curve = ql.PiecewiseLogLinearDiscount(ql_trade_date, isda_rate_helpers, ql.Actual365Fixed())
discount_curve = ql.YieldTermStructureHandle(rate_curve)

In [123]:
cds_schedule = ql.Schedule(
    ql_trade_date, 
    ql_term_date,
    3*ql.Period(ql.Monthly),
    ql.WeekendsOnly(),
    ql.Following, 
    ql.Unadjusted,
    ql.DateGeneration.CDS,
    False
)

In [124]:
quoted_trade = ql.CreditDefaultSwap(
    ql.Protection.Buyer,  # side
    10000000,  # nominal
    0,  # upfront
    cds_price['last_price'] / 10000,  # spread
    cds_schedule,  # Schedule
    ql.Following, 
    ql.Actual360(), 
    True, 
    True, 
    ql_trade_date,
    ql_upfront_date, 
    ql.FaceValueClaim(), 
    ql.Actual360(True),
)
# quoted_trade.setPricingEngine(engine)

In [125]:
h = quoted_trade.impliedHazardRate(
    0, 
    discount_curve, 
    ql.Actual365Fixed(),
    cds_ref['recovery'], 
    1e-10,
    ql.CreditDefaultSwap.ISDA
)

In [126]:
fhr_curve = ql.FlatHazardRate(
    0, 
    ql.WeekendsOnly(),
    ql.QuoteHandle(ql.SimpleQuote(h)),
    ql.Actual365Fixed()
)
pd_curve = ql.DefaultProbabilityTermStructureHandle(fhr_curve)

In [127]:
engine = ql.IsdaCdsEngine(pd_curve, cds_ref['recovery'], discount_curve)

In [128]:
conventional_trade = ql.CreditDefaultSwap(
    ql.Protection.Buyer,  # side
    10000000,  # notional
    0,  # upfront
    100 / 10000,  # spread
    cds_schedule,  # schedule
    ql.Following,  # paymentConvention
    ql.Actual360(),  # dayCounter
    True,  # settlesAccrual
    True,  # paysAtDefaultTime
    ql_trade_date,  # protectionStart
    ql_upfront_date,  # upfrontDate
    ql.FaceValueClaim(),  # Claim
    ql.Actual360(True),  # lastPeriodDayCounter
    True,  # rebatesAccrual
)
conventional_trade.setPricingEngine(engine)

In [129]:
cds_ref

security    ITALY CDS USD SR 5Y D14 Curncy
maturity               2027-06-20 00:00:00
recovery                               0.4
Name: 0, dtype: object

In [130]:
cds_price

date                     2022-06-24 00:00:00
security      ITALY CDS USD SR 5Y D14 Curncy
last_price                             134.0
Name: 0, dtype: object

In [131]:
f"NPV: {conventional_trade.NPV()}, Price: {100 * (1+conventional_trade.fairUpfront())}"

'NPV: 151075.48244529436, Price: 98.48893250620645'

In [ ]:
quoted_trade.coupons()

In [ ]:
ql.CashFlow.amount(conventional_trade.coupons())

In [ ]:
conventional_trade.couponLegNPV()

In [ ]:
conventional_trade.defaultLegNPV()

In [ ]:
conventional_trade.NPV()

In [ ]:
conventional_trade.accrualRebateNPV()

In [ ]:
1+conventional_trade.fairUpfront()

In [ ]:
quoted_trade.fairUpfront()


In [ ]:
probabilityCurve = ql.RelinkableDefaultProbabilityTermStructureHandle()

In [ ]:

termDate = ql.Date(20, 6, 2010),
             ql.Date(20, 6, 2011),
             ql.Date(20, 6, 2012),
             ql.Date(20, 6, 2016),
             ql.Date(20, 6, 2019)

spreads = [0.001, 0.1]
recoveries = [0.2, 0.4]

markitValues = [97798.29358,  # 0.001
                97776.11889,  # 0.001
                -914971.5977,  # 0.1
                -894985.6298,  # 0.1
                186921.3594,  # 0.001
                186839.8148,  # 0.001
                -1646623.672,  # 0.1
                -1579803.626,  # 0.1
                274298.9203,
                274122.4725,
                -2279730.93,
                -2147972.527,
                592420.2297,
                591571.2294,
                -3993550.206,
                -3545843.418,
                797501.1422,
                795915.9787,
                -4702034.688,
                -4042340.999]

tolerance = 1.0e-2

l = 0
distance = 0

data = []
upfront_date = ql.WeekendsOnly().advance(trade_date, 3 * ql.Period(ql.Daily))
for termDate in termDates:
    for spread in spreads:
        for recovery in recoveries:
            cdsSchedule = ql.Schedule(
                trade_date, 
                termDate,
                3 * ql.Period(ql.Monthly),
                ql.WeekendsOnly(),
                ql.Following, 
                ql.Unadjusted,
                ql.DateGeneration.CDS,
                False
            )

            quotedTrade = ql.CreditDefaultSwap(
                ql.Protection.Buyer, 
                10000000, 
                0, 
                spread, 
                cdsSchedule,
                ql.Following, ql.Actual360(), True, True, trade_date,
                upfront_date, ql.FaceValueClaim(), ql.Actual360(True))

            h = quotedTrade.impliedHazardRate(
                0, 
                discountCurve, 
                ql.Actual365Fixed(),
                recovery, 
                1e-10,
                ql.CreditDefaultSwap.ISDA
            )

            probabilityCurve.linkTo(
                ql.FlatHazardRate(0, ql.WeekendsOnly(),
                                  ql.QuoteHandle(ql.SimpleQuote(h)),
                                  ql.Actual365Fixed()))

            engine = ql.IsdaCdsEngine(probabilityCurve, recovery, discountCurve)
            conventionalTrade = ql.CreditDefaultSwap(
                ql.Protection.Buyer, 10000000, 0, 0.01, cdsSchedule,
                ql.Following, ql.Actual360(), True, True, trade_date,
                upfront_date, ql.FaceValueClaim(), ql.Actual360(True))
            conventionalTrade.setPricingEngine(engine)

            upfront = conventionalTrade.notional() * conventionalTrade.fairUpfront()

            data.append(
                (termDate,
                 spread,
                 recovery,
                 h,
                 upfront,
                 markitValues[l],
                 abs(upfront - markitValues[l]),
                 abs(upfront - markitValues[l]) < tolerance)
            )
            distance = distance + abs(upfront - markitValues[l])

            l = l + 1

df = pd.DataFrame(data, columns=["Term date", "Spread", "Recovery",
                                 "Hazard rate", "Upfront", "Markit value", "Distance", "Within tolerance"])


df.style.format({'Spread': '{:.4%}', 'Hazard rate': '{:.2%}', 'Upfront': '{:.2f}',
                 'Markit value': '{:.2f}', 'Distance': '{:.6f}'})

In [ ]:
probabilityCurve = ql.RelinkableDefaultProbabilityTermStructureHandle()

termDates = [ql.Date(20, 6, 2010),
             ql.Date(20, 6, 2011),
             ql.Date(20, 6, 2012),
             ql.Date(20, 6, 2016),
             ql.Date(20, 6, 2019)]

spreads = [0.001, 0.1]
recoveries = [0.2, 0.4]

markitValues = [97798.29358,  # 0.001
                97776.11889,  # 0.001
                -914971.5977,  # 0.1
                -894985.6298,  # 0.1
                186921.3594,  # 0.001
                186839.8148,  # 0.001
                -1646623.672,  # 0.1
                -1579803.626,  # 0.1
                274298.9203,
                274122.4725,
                -2279730.93,
                -2147972.527,
                592420.2297,
                591571.2294,
                -3993550.206,
                -3545843.418,
                797501.1422,
                795915.9787,
                -4702034.688,
                -4042340.999]

tolerance = 1.0e-2

l = 0
distance = 0

data = []
upfront_date = ql.WeekendsOnly().advance(trade_date, 3 * ql.Period(ql.Daily))
for termDate in termDates:
    for spread in spreads:
        for recovery in recoveries:
            cdsSchedule = ql.Schedule(
                trade_date, 
                termDate,
                3 * ql.Period(ql.Monthly),
                ql.WeekendsOnly(),
                ql.Following, 
                ql.Unadjusted,
                ql.DateGeneration.CDS,
                False
            )

            quotedTrade = ql.CreditDefaultSwap(
                ql.Protection.Buyer, 
                10000000, 
                0, 
                spread, 
                cdsSchedule,
                ql.Following, ql.Actual360(), True, True, trade_date,
                upfront_date, ql.FaceValueClaim(), ql.Actual360(True))

            h = quotedTrade.impliedHazardRate(
                0, 
                discountCurve, 
                ql.Actual365Fixed(),
                recovery, 
                1e-10,
                ql.CreditDefaultSwap.ISDA
            )

            probabilityCurve.linkTo(
                ql.FlatHazardRate(0, ql.WeekendsOnly(),
                                  ql.QuoteHandle(ql.SimpleQuote(h)),
                                  ql.Actual365Fixed()))

            engine = ql.IsdaCdsEngine(probabilityCurve, recovery, discountCurve)
            conventionalTrade = ql.CreditDefaultSwap(
                ql.Protection.Buyer, 10000000, 0, 0.01, cdsSchedule,
                ql.Following, ql.Actual360(), True, True, trade_date,
                upfront_date, ql.FaceValueClaim(), ql.Actual360(True))
            conventionalTrade.setPricingEngine(engine)

            upfront = conventionalTrade.notional() * conventionalTrade.fairUpfront()

            data.append(
                (termDate,
                 spread,
                 recovery,
                 h,
                 upfront,
                 markitValues[l],
                 abs(upfront - markitValues[l]),
                 abs(upfront - markitValues[l]) < tolerance)
            )
            distance = distance + abs(upfront - markitValues[l])

            l = l + 1

df = pd.DataFrame(data, columns=["Term date", "Spread", "Recovery",
                                 "Hazard rate", "Upfront", "Markit value", "Distance", "Within tolerance"])


df.style.format({'Spread': '{:.4%}', 'Hazard rate': '{:.2%}', 'Upfront': '{:.2f}',
                 'Markit value': '{:.2f}', 'Distance': '{:.6f}'})

In [ ]:
print('total distance:', distance)